In [ ]:
# 403 error .giving error of 403

In [3]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  
site = 'devex_com_publication'
site_name = 'Vietnam Institute for Economic Policy Research (VERP) (Vietnam)'
c = Crawl()  

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_fetch_cat_url = 0
publish_source = 'devex.com'
country = 'Vietnam'
language = 'English'
images_path = []
cookies = {
    'ua_session_id': 'f5def67311b8ef25c87addacdb3ea7b674d82c9aaaca8677f72d76f26ad71798',
    '_frontend_session': 'd43a7b6252b9eae567c6c92d22ae6577',
    '_mkto_trk': 'id:685-KBL-765&token:_mch-devex.com-1622718164437-10373',
    'datadome': '5RONmcLFO-sEeIX4_d1wZ0FzJU-atIwNsMnCBuVPCPpAAiHWP7J8rUeH-e.Jyg~_27v4dKyLiLYviFg~mN.XnV7U5QbyNSGGnUN.ov7b.T',
    '_omappvp': 'WA62Hf0mRGhLAJESunkAZqGf2PzU1u4deN0LbWLUTgBRYAc51QjkwUqdndGIVT4XbWPoYNmLR8tkXk8cYg2q3HvXGOFMdpHI',
    'AMP_TOKEN': '%24NOT_FOUND',
    '_fbp': 'fb.1.1622718168695.242009328',
    '_ga': 'GA1.2.34423317.1622718170',
    '_gid': 'GA1.2.1708202018.1622718170',
    'om-793654': 'true',
    '__atuvc': '1%7C22',
    '__atuvs': '60b8c2c0bdc83740000',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0',
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://www.devex.com/news/search',
    'X-CSRF-Token': 'qxBDqtuvawz/FxhaOXs2OrSFFkx6XgEoxP2JX3huYWRg7a79G1l9Xe5McZpvB49ms/0oczp8LE2H9YRA7EYeDQ==',
    'X-Requested-With': 'XMLHttpRequest',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
}

params = (
    ('page[size]', '80'),
)

response = requests.get('https://www.devex.com/api/public/search/articles', headers=headers, params=params, cookies=cookies).text

for i in response.split('"slug_and_id"')[1:]:
    # source_link 
    source_link = c.scrap('"(.*?)"' , i)   
    if 'http' not in source_link:
        source_link = 'https://www.devex.com/news/' + source_link
    # handle duplicates
    source_link_query = {'source_link':source_link}
    dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
    if dic:
        duplicate_data += 1
        continue

    time.sleep(random.randint(1,3))

    logger.info(f'Fetching {source_link}\n')

    page = c.download_page(source_link)   
    if page.startswith('Unable to fetch'):
        logger.info(page)
        unable_to_fetch_article_url += 1
        continue    

    source_headline = c.scrap('<title>(.*?)\|', page)

    # skip if headline not found
    if not source_headline:
        logger.info(f'Skipping due to headline {source_link}\n')
        skipped_due_to_headline += 1
        continue

     # Date and time
    pub_date, publish_time = '', current_time

    try:   
        date_time_str = c.scrap('publication"\s*:"(.*?)\.', page)
        date_time_str = re.sub('&.*?;','',date_time_str)
        date_time_str = re.sub('[^\w+]', '', date_time_str)  
        date_time_obj = datetime.strptime(date_time_str, '%Y%m%dT%H%M%S')
        ist_date_time = date_time_obj - timedelta(hours = 1,minutes = 30)  
        ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
        pub_date = ist_date_time[:10]
        publish_time = ist_date_time[11:]
    
    except:
        pass

    # skip null date
    if not pub_date:
        logger.info(f'Skipping due to date {source_link}\n')            
        skipped_due_to_date += 1
        continue

    # break if date is not today's date
    if pub_date != created_on:
        break    

    # source_content 
    source_content = c.scrap('</figcaption></figure><p>(.*?)</p></div><div\s*class="clearfix">',page)
    if not source_content:
        source_content = c.scrap('</figcaption></figure><p>(.*?)</p><p><strong><em>',page)
    if not source_content:
        source_content = c.scrap ('</figcaption></figure><p>(.*?)</p><figure>',page)
    if not source_content:
        source_content = c.scrap('</figcaption></figure><p>(.*?)<div\s*class="subscribe-to-panel">',page)   
    if not source_content:
        source_content = c.scrap('</p></div><p>(.*?)</p><figure>',page)
    if not source_content:
        source_content = c.scrap('</p></div><p>(.*?)</p><p><strong><em>',page)
    if not source_content:
        source_content = c.scrap('</p></div><p>(.*?)</p><div\s*class="adbutler"',page)
    if not source_content:
        source_content = c.scrap('<div\s*class="article-teaser">(.*?)</p><a\s*class="btn\s*btn-lg\s*btn-solid-gray-dark',page)
    source_content = re.sub('<span\s*style=(.*?)</span>','',source_content,re.S)
    source_content = re.sub('&.*?;','',source_content,re.S)
    source_content = c.strip_html(source_content)
  
    if not source_content:
        logger.info(f'Skipping due to content {source_link}\n')            
        skipped_due_to_content += 1
        continue


    journalist =c.scrap("'author-name':'(.*?)'",page)

    if not journalist: journalist = 'NA'

    # current date and time 00
    harvest_time = datetime.now().strftime("%H:%M:%S")

    # temp link
    temp_link = source_link

    # headline and content 
    headline = source_headline
    content = source_content

    # overall_tonality
    overall_tonality = ''

    # word count
    word_count = len((source_headline + ' ' + source_content).split())

    html_content = ''

    # image_urls
    image_urls = []
    

    # storing the above data in a dictionary
    clientdata ={
                    "client_master" : client_id, 
                    "articleid":client_id,
                    "medium":'Web' ,
                    "searchkeyword":[],
                    "entityname" : [] ,
                    "process_flage":"1",
                    "na_flage":"0",
                    "na_reason":"",
                    "qc_by":"",
                    "qc_on":"",
                    "location":"",
                    "spokeperson":"",
                    "quota":"",
                    "overall_topics":"",
                    "person":"",
                    "overall_entites":"",
                    "overall_tonality": overall_tonality,
                    "overall_wordcount":word_count,
                    "article_subjectivity":"",
                    "article_summary":"",
                    "pub_date":pub_date,
                    "publish_time":publish_time,
                    "harvest_time":harvest_time,
                    "temp_link":temp_link,
                    "publish_source": publish_source,
                    "programme":'null',
                    "feed_class":"News",
                    "publishing_platform":"",
                    "klout_score":"",
                    "journalist":journalist,
                    "headline":headline,
                    "content":content,
                    "source_headline":source_headline,
                    "source_content":source_content,
                    "language":language,
                    "presence":'null',
                    "clip_type":'null',
                    "prog_slot":'null',
                    "op_ed":'0',
                    "location_mention":'',
                    "source_link":source_link,
                    "author_contact":'',
                    "author_emailid":'',
                    "author_url":'',
                    "city":'',
                    "state":'',
                    "country":country,
                    "source":publish_source,
                    "foot_fall":'',
                    "created_on":created_on,
                    "active":'1',
                    'crawl_flag':2,
                    "images_path":images_path,
                    "html_content":html_content,
                
                }

#     cl_data.insert_one(clientdata)  
    no_of_data += 1
logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'country: {country}\n')
logger.info(f'language: {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n') 